In [2]:
import numpy as np
import os
import string

# Notebook for running the densmap calculations of the different systems

In [4]:
systems = ['IA','PA', 'FA_G-prot','FA_no_G-prot']

unique_lipids = ['POPC','POPS','POPE','POPA','POPG','CHOL','DPG3', 'DPSM', 'PAP6']

all_lipids_sele = " ".join(['{0:s}'.format(i) for i in unique_lipids ])

lipid_dir = {"FA_G-prot":"POPC, POPS, POPA, POPG, POPE, CHOL, DPGM3, DPSM, PAP6",
            "FA_no_G-prot":"POPC, POPS, POPA, POPG, POPE, CHOL, DPGM3, DPSM, PAP6",
            "IA":"POPC, POPS, POPA, POPG, POPE, CHOL, DPGM3, DPSM, PAP6",
            "PA":"POPC, POPS, POPA, POPG, POPE, CHOL, DPGM3, DPSM, PAP6"}

begin_time_ps=["19000000"]


### fit trajectorie

for s in systems:
    for r in range(3):
        xtc = "XTC/{0:s}_{1:d}_nowat_res_center.xtc".format(s,r)
        tpr = "ORIGO_TPR/FA_G-prot_0_DENSMAP.tpr"
        ndx = "NDX/{0:2}_{1:d}_nowat_rdf_chains.ndx".format(s,r)
        out = "XTC/{0:s}_{1:d}_nowat_res_center_densmap_fit.xtc".format(s,r)
        os.system("echo 5 > sel")
        os.system("echo 8 >> sel")
        if os.path.exists(out):
            print ("{0:s} exists".format(out))
            continue
        else:
            os.system("gmx_s trjconv -f {0:s} -s {1:s} -n {2:s} -fit rot+trans -o {3:s}".format(xtc,tpr,ndx,out))

# Done it in a script - look there

# Fit trjactories : on same reference structure - count densities lst 0.5 µs

In [30]:
Head = False #Set True if only considering the headgroups, otherwise False and consider the entire lipid
## FITTET XTC

for idx, s in enumerate(systems):
    for r in range(3):
        print ("Replica", r)
        #b = begin_time_ps[idx]
        #e = end_time_ps[idx]
        tpr = "TPR/{0:s}_{1:d}_nowat.tpr".format(s,r)
        ndx = "NDX/{0:s}_{1:d}_nowat.ndx".format(s,r)
        xtc = "XTC/{0:s}_{1:d}_nowat_res_center_fit.xtc".format(s,r)
        lipids = lipid_dir[s]
        if Head == True:
            sel = [ "{0:s}_&_PO4".format(l) for l in lipids.split(',') ]
            output_base = "DENSMAP/FIT/COUNT/Headgroups/{0:s}_{1:d}_nowat".format(s,r)
        else:
            sel = [ "{0:s}".format(l) for l in lipids.split(',') ]
            output_base = "DENSMAP/FIT/COUNT/{0:s}_{1:d}_nowat".format(s,r)
        for sele in sel:
            print ("lipid {0:s}".format(sele))
            output_xpm = output_base+"_{0:s}_densmap.xpm".format(sele).translate({ord(c): None for c in string.whitespace})
            output_dat = output_base+"_{0:s}_densmap.dat".format(sele).translate({ord(c): None for c in string.whitespace})
            output_eps = output_base+"_{0:s}_densmap.eps".format(sele).translate({ord(c): None for c in string.whitespace})
            if os.path.exists(output_dat):
                print ("{0:s} exists".format(output_dat))
                continue
            else:
                grep_cmd = "grep '\[' "+ndx+"| gawk '{print NR-1, $2}'"+"| grep '{0:s}' |".format(sele)+"gawk '{print $1}' > sel"
                print (grep_cmd)
                os.system(grep_cmd)
                densmap_cmd1 = "cat sel | gmx_s densmap -f {0:s} -s {1:s} -n {2:s} -o {3:s} -dt 10 -b 19000000 -unit count".format(xtc, tpr, ndx, output_xpm, b, e)
                print (densmap_cmd1)
                os.system(densmap_cmd1)
                densmap_cmd2 = "cat sel | gmx_s densmap -f {0:s} -s {1:s} -n {2:s} -od {3:s} -dt 10 -b 19000000 -unit count".format(xtc, tpr, ndx, output_dat, b, e)
                print (densmap_cmd2)
                os.system(densmap_cmd2)

Replica 0
lipid POPC
DENSMAP/FIT/COUNT/5XEZ_LB1_0_nowat_POPC_densmap.dat exists
lipid  POPS
DENSMAP/FIT/COUNT/5XEZ_LB1_0_nowat_POPS_densmap.dat exists
Replica 1
lipid POPC
DENSMAP/FIT/COUNT/5XEZ_LB1_1_nowat_POPC_densmap.dat exists
lipid  POPS
DENSMAP/FIT/COUNT/5XEZ_LB1_1_nowat_POPS_densmap.dat exists
Replica 2
lipid POPC
DENSMAP/FIT/COUNT/5XEZ_LB1_2_nowat_POPC_densmap.dat exists
lipid  POPS
DENSMAP/FIT/COUNT/5XEZ_LB1_2_nowat_POPS_densmap.dat exists
Replica 0
lipid POPC
DENSMAP/FIT/COUNT/5XEZ_LB2_0_nowat_POPC_densmap.dat exists
lipid  POPE
DENSMAP/FIT/COUNT/5XEZ_LB2_0_nowat_POPE_densmap.dat exists
Replica 1
lipid POPC
grep '\[' NDX/5XEZ_LB2_1_nowat.ndx| gawk '{print NR-1, $2}'| grep 'POPC' |gawk '{print $1}' > sel
cat sel | gmx_s densmap -f XTC/5XEZ_LB2_1_nowat_mol_center_fit_densmap.xtc -s TPR/5XEZ_LB2_1_nowat.tpr -n NDX/5XEZ_LB2_1_nowat.ndx -o DENSMAP/FIT/COUNT/5XEZ_LB2_1_nowat_POPC_densmap.xpm -dt 10 -b 14000000 -e 14500000 -unit count
cat sel | gmx_s densmap -f XTC/5XEZ_LB2_1_nowat_

Head = True #Set True if only considering the headgroups, otherwise False and consider the entire lipid
## FITTET XTC

for idx, s in enumerate(systems):
    for r in range(3):
        print ("Replica", r)
        tpr = "TPR/{0:s}_{1:d}_nowat.tpr".format(s,r)
        ndx = "NDX/{0:s}_{1:d}_nowat.ndx".format(s,r)
        xtc = "XTC/{0:s}_{1:d}_nowat_res_center_fit.xtc".format(s,r)
        lipids = lipid_dir[s]
        if Head == True:
            sel = [ "{0:s}_&_PO4".format(l) for l in lipids.split(',') ]
            output_base = "DENSMAP/FIT/COUNT/Headgroups/{0:s}_{1:d}_nowat".format(s,r)
        else:
            sel = [ "{0:s}".format(l) for l in lipids.split(',') ]
            output_base = "DENSMAP/FIT/COUNT/{0:s}_{1:d}_nowat".format(s,r)
        for sele in sel:
            print ("lipid {0:s}".format(sele))
            output_xpm = output_base+"_{0:s}_densmap.xpm".format(sele).translate({ord(c): None for c in string.whitespace})
            output_dat = output_base+"_{0:s}_densmap.dat".format(sele).translate({ord(c): None for c in string.whitespace})
            output_eps = output_base+"_{0:s}_densmap.eps".format(sele).translate({ord(c): None for c in string.whitespace})
            if os.path.exists(output_dat):
                print ("{0:s} exists".format(output_dat))
                continue
            else:
                grep_cmd = "grep '\[' "+ndx+"| gawk '{print NR-1, $2}'"+"| grep '{0:s}' |".format(sele)+"gawk '{print $1}' > sel"
                print (grep_cmd)
                os.system(grep_cmd)
                densmap_cmd1 = "cat sel | gmx densmap -f {0:s} -s {1:s} -n {2:s} -o {3:s} -dt 10 -b 19000000 -unit count".format(xtc, tpr, ndx, output_xpm, b, e)
                print (densmap_cmd1)
                os.system(densmap_cmd1)
                densmap_cmd2 = "cat sel | gmx densmap -f {0:s} -s {1:s} -n {2:s} -od {3:s} -dt 10 -b 19000000 -unit count".format(xtc, tpr, ndx, output_dat, b, e)
                print (densmap_cmd2)
                os.system(densmap_cmd2)

# Protein Density

In [33]:

## FITTET XTC

for idx, s in enumerate(systems):
    for r in range(3):
        print ("Replica", r)
        tpr = "TPR/{0:s}_{1:d}_nowat.tpr".format(s,r)
        ndx = "NDX/{0:s}_{1:d}_nowat.ndx".format(s,r)
        xtc = "XTC/{0:s}_{1:d}_nowat_res_center_fit.xtc".format(s,r)
        output_base = "DENSMAP/FIT/Protein/{0:s}_{1:d}_nowat".format(s,r)
        sele='Protein'
        output_xpm = output_base+"_{0:s}_densmap.xpm".format(sele)
        output_dat = output_base+"_{0:s}_densmap.dat".format(sele)
        output_eps = output_base+"_{0:s}_densmap.eps".format(sele)
        if os.path.exists(output_dat):
            print ("{0:s} exists".format(output_dat))
            continue
        else:
            grep_cmd = "grep '\[' "+ndx+"| gawk '{print NR-1, $2}'"+"| grep '{0:s}' |".format(sele)+"gawk '{print $1}' > sel"
            print (grep_cmd)
            os.system(grep_cmd)
            densmap_cmd1 = "cat sel | gmx densmap -f {0:s} -s {1:s} -n {2:s} -o {3:s} -dt 10 -b 19000000 -unit count".format(xtc, tpr, ndx, output_xpm, b, e)
            print (densmap_cmd1)
            os.system(densmap_cmd1)
            densmap_cmd2 = "cat sel | gmx densmap -f {0:s} -s {1:s} -n {2:s} -od {3:s} -dt 10 -b 19000000 -unit count".format(xtc, tpr, ndx, output_dat, b, e)
            print (densmap_cmd2)
            os.system(densmap_cmd2)

Replica 0
DENSMAP/FIT/Protein/5XEZ_LB1_0_nowat_Protein_densmap.dat exists
Replica 1
DENSMAP/FIT/Protein/5XEZ_LB1_1_nowat_Protein_densmap.dat exists
Replica 2
DENSMAP/FIT/Protein/5XEZ_LB1_2_nowat_Protein_densmap.dat exists
Replica 0
DENSMAP/FIT/Protein/5XEZ_LB2_0_nowat_Protein_densmap.dat exists
Replica 1
grep '\[' NDX/5XEZ_LB2_1_nowat.ndx| gawk '{print NR-1, $2}'| grep 'Protein' |gawk '{print $1}' > sel
cat sel | gmx densmap -f XTC/5XEZ_LB2_1_nowat_mol_center_fit_densmap.xtc -s TPR/5XEZ_LB2_1_nowat.tpr -n NDX/5XEZ_LB2_1_nowat.ndx -o DENSMAP/FIT/Protein/5XEZ_LB2_1_nowat_Protein_densmap.xpm -dt 10 -b 14000000 -e 14500000 -unit count
cat sel | gmx densmap -f XTC/5XEZ_LB2_1_nowat_mol_center_fit_densmap.xtc -s TPR/5XEZ_LB2_1_nowat.tpr -n NDX/5XEZ_LB2_1_nowat.ndx -od DENSMAP/FIT/Protein/5XEZ_LB2_1_nowat_Protein_densmap.dat -dt 10 -b 14000000 -e 14500000 -unit count
Replica 2
DENSMAP/FIT/Protein/5XEZ_LB2_2_nowat_Protein_densmap.dat exists
Replica 0
DENSMAP/FIT/Protein/5XEZ_LB3_0_nowat_Protein